## Problema del Viajante de Comercio o TSP

$$\min \sum_{a \in A} c_{a}x_{a}$$

$$\sum_{a \in \delta^+ (i)} x_{a} = \sum_{a \in \delta^- (i)} s_{a} = 1$$

$$\sum_{a \in A(S)} x_{a} \leq |S| - 1$$

$$\forall S \subset N$$

$$0 \leq x_{a} \leq 1$$

$$x_{a} \in \mathbb{Z}$$

Primero importamos la librería de optimi ... y numpy para la representación del grafo que conecta los caminos 

In [7]:
from pulp import *
import numpy as np 

In [8]:
num_cities = 5
distances = np.array([
  [0, 1, 1, 1, 1],
  [1, 0, 1, -1, 12],
  [1, 1, 0, -4, 42],
  [1, 3, 0, 0, 1],
  [1, -2, -1, 1, 0]
])

prob = LpProblem(
  name="TSP",
  sense=LpMinimize
)

In [9]:
# Decision variables: x[i][j] is 1 if the path goes from city i to city j, 0 otherwise
x = LpVariable.dicts("x", (range(num_cities), range(num_cities)), cat='Binary')

# Objective function: minimize the total distance
prob += lpSum(distances[i][j] * x[i][j] for i in range(num_cities) for j in range(num_cities))

# Constraints: each city is visited exactly once
for i in range(num_cities):
  prob += lpSum(x[i][j] for j in range(num_cities) if i != j) == 1
  prob += lpSum(x[j][i] for j in range(num_cities) if i != j) == 1

# Subtour elimination constraints
u = LpVariable.dicts("u", range(num_cities), lowBound=0, cat='Continuous')
for i in range(1, num_cities):
  for j in range(1, num_cities):
    if i != j:
      prob += u[i] - u[j] + num_cities * x[i][j] <= num_cities - 1

prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/zkorpion/Projects/python-vault/_env/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/e0d81a24be3e4055a7f55023480531a8-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/e0d81a24be3e4055a7f55023480531a8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 27 COLUMNS
At line 163 RHS
At line 186 BOUNDS
At line 207 ENDATA
Problem MODEL has 22 rows, 24 columns and 76 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is -3.6 - 0.00 seconds
Cgl0004I processed model has 22 rows, 24 columns (20 integer (20 of which binary)) and 76 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 4 integers unsatisfied sum - 1.6
Cbc0038I Pass   1: suminf.    1.60000 (8) obj. -2.6 iterations 6
Cbc0038I Pass   2: suminf.    0.00000 (0) obj. 3 iterations 

1

In [11]:
print("Path:")
for i in range(num_cities):
  for j in range(num_cities):
    if value(x[i][j]) == 1:
      print(f"City {i+1} -> City {j+1}")

Path:
City 1 -> City 3
City 2 -> City 1
City 3 -> City 4
City 4 -> City 5
City 5 -> City 2
